In [36]:
from openai import OpenAI
import banidb
from pinecone import Pinecone
import re

pc = Pinecone(api_key="")

client = OpenAI(api_key="")



In [37]:
def get_embedding(text, model="text-embedding-3-small"):
  #  text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding


In [38]:
index = pc.Index("bani-db-serverless")

In [39]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1499},
                'pankti_combined': {'vector_count': 2525},
                'punjabi-and-english-base': {'vector_count': 2},
                'punjabi-and-english-bdb-steek': {'vector_count': 499},
                'punjabi-and-english-separate-then-combined': {'vector_count': 1498},
                'punjabi-and-english-verses-combined': {'vector_count': 28}},
 'total_vector_count': 6051}

In [59]:
banidb.shabad(41232)
# banidb.random('N')

{'shabad_id': 41232,
 'source_uni': 'ਭਾਈ ਗੁਰਦਾਸ ਜੀ ਵਾਰਾਂ',
 'source_eng': 'Bhai Gurdas Ji Vaaran',
 'writer': 'Bhai Gurdaas Ji',
 'ang': None,
 'verses': [{'verse_id': 208574,
   'verse': 'ਪਸੂ ਖੜ ਖਾਤ ਖਲ ਸਬਦ ਸੁਰਤਿ ਹੀਨ ਮੋਨ ਕੋ ਮਹਾਤਮ ਪੈ ਅੰਮ੍ਰਿਤ ਪ੍ਰਵਾਹ ਜੀ ॥',
   'steek': {'en': {'bdb': "An animal eats green grass and hay. He is bereft of all knowledge of Lord's word. Due to his inability to speak, gives nectar-like milk."},
    'pu': {'ss': {'gurmukhi': None, 'unicode': None},
     'ft': {'gurmukhi': '', 'unicode': ''},
     'bdb': {'gurmukhi': None, 'unicode': None}},
    'es': {'sn': None},
    'hi': {}},
   'transliteration': {'english': 'pasoo khaR khaat khal sabadh surat heen mon ko mahaatam pai a(n)mirat pravaeh jee ||',
    'hindi': 'पसू खड़ खात खल सबद सुरति हीन मोन को महातम पै अंमृत प्रवाह जी ॥',
    'en': 'pasoo khaR khaat khal sabadh surat heen mon ko mahaatam pai a(n)mirat pravaeh jee ||',
    'hi': 'पसू खड़ खात खल सबद सुरति हीन मोन को महातम पै अंमृत प्रवाह जी ॥',
    'ipa': 'pəs

In [41]:
def get_full_shabad(shabad_id):
    # return shabad in eng and punjabi
    try:
        shabad_dict = banidb.shabad(shabad_id)
    except KeyError as ke:
        print("shabad_id does not exist")
        return None
    
    shabad_text_pa = ""
    shabad_text_eng = ""
    for verse in shabad_dict['verses']:
        shabad_text_pa += verse['verse'] 
        if verse["steek"]["en"]["bdb"] is not None:
            shabad_text_eng += verse["steek"]["en"]["bdb"]  #multiple english steeks available, choosing bdb
        else:
            shabad_text_eng += " "
    return {"shabad_text_pa":shabad_text_pa,
            "shabad_text_eng":shabad_text_eng,
            "shabad_id":shabad_dict["shabad_id"]}

# 41000,41711 Bhai Gurdas Singh Ji Vaars
# 32001, 32525 Bhai Nand Lal ji Vaars
# shabad id -7402 dasam bani start - ~12800
# B, till page 40 - 
# D, till ang 1428- Dasam bani

In [57]:
full_shabad = get_full_shabad(7956)
full_shabad['shabad_text_pa'], full_shabad['shabad_text_eng']

('ਭਰਥ ਬਾਚ ਰਾਮ ਪ੍ਰਤਿ ॥ਕੰਠ ਅਭੂਖਨ ਛੰਦ ॥ਜਾਉ ਕਹਾ ਪਗ ਭੇਟ ਕਹਉ ਤੁਹ ॥ਲਾਜ ਨ ਲਾਗਤ ਰਾਮ ਕਹੋ ਮੁਹ ॥ਮੈ ਅਤਿ ਦੀਨ ਮਲੀਨ ਬਿਨਾ ਗਤ ॥ਰਾਖ ਲੈ ਰਾਜ ਬਿਖੈ ਚਰਨਾਮ੍ਰਿਤ ॥੨੮੭॥ਚੱਛ ਬਿਹੀਨ ਸੁੱਪਛ ਜਿਮੰ ਕਰ ॥ਤਿਉ ਪ੍ਰਭ ਤੀਰ ਗਿਰਯੋ ਪਗ ਭਰਥਰ ॥ਅੰਕ ਰਹੇ ਗਹ ਰਾਮ ਤਿਸੈ ਤਬ ॥ਰੋਇ ਮਿਲੇ ਲਛਨਾਦਿ ਭੱਯਾ ਸਭ ॥੨੮੮॥ਪਾਨਿ ਪੀਆਇ ਜਗਾਇ ਸੁ ਬੀਰਹ ॥ਫੇਰਿ ਕਹਯੋ ਹਸ ਸ੍ਰੀ ਰਘੁਬੀਰਹ ॥ਤ੍ਰਿਯੋਦਸ ਬਰਖ ਗਏ ਫਿਰਿ ਐਹੈ ॥ਜਾਹੁ ਹਮੈ ਕਛੁ ਕਾਜ ਕਿਵੈਹੈ ॥੨੮੯॥ਚੀਨ ਗਏ ਚਤੁਰਾ ਚਿਤ ਮੋ ਸਭ ॥ਸ੍ਰੀ ਰਘੁਬੀਰ ਕਹੀ ਅਸ ਕੈ ਜਬ ॥ਮਾਤ ਸਮੋਧ ਸੁ ਪਾਵਰਿ ਲੀਨੀ ॥ਅਉਰ ਬਸੇ ਪੁਰ ਅਉਧ ਨ ਚੀਨੀ ॥੨੯੦॥ਸੀਸ ਜਟਾਨ ਕੋ ਜੂਟ ਧਰੇ ਬਰ ॥ਰਾਜ ਸਮਾਜ ਦੀਯੋ ਪਊਵਾ ਪਰ ॥ਰਾਜ ਕਰੇ ਦਿਨੁ ਹੋਤ ਉਜਿਆਰੈ ॥ਰੈਨਿ ਭਏ ਰਘੁਰਾਜ ਸੰਭਾਰੈ ॥੨੯੧॥ਜੱਜਰ ਭਯੋ ਝੁਰ ਝੰਝਰ ਜਿਉ ਤਨ ॥ਰਾਖਤ ਸ੍ਰੀ ਰਘੁਰਾਜ ਬਿਖੈ ਮਨ ॥ਬੈਰਿਨ ਕੇ ਰਨ ਬਿੰਦ ਨਿਕੰਦਤ ॥ਭਾਖਤ ਕੰਠਿ ਅਭੂਖਨ ਛੰਦਤ ॥੨੯੨॥',
 'Speech of Bharat addressed to Ram :KANTH AABHUSHAN STANZA“O Ram! where should I go now after touching your feet? Shall I not be ashmed?“I am extremely low, dirty and motionless. O Ram! manage your kingdom and glorify it with your ambrosial feet.”287.Just as a bird become sightless fells down, in the same way Bharat fell down before Ram.A

In [90]:


# Define the regex pattern
pattern = r'(॥[੦੧੨੩੪੫੬੭੮੯]+॥)'

# Split the Punjabi text based on the regex pattern
chunks = re.split(pattern, get_full_shabad(1300)["shabad_text_pa"])

# Print the resulting chunks
for chunk in chunks:
    print(chunk)


ਗਉੜੀ ਕਬੀਰ ਜੀ ॥ਚੋਆ ਚੰਦਨ ਮਰਦਨ ਅੰਗਾ ॥ਸੋ ਤਨੁ ਜਲੈ ਕਾਠ ਕੈ ਸੰਗਾ 
॥੧॥
ਇਸੁ ਤਨ ਧਨ ਕੀ ਕਵਨ ਬਡਾਈ ॥ਧਰਨਿ ਪਰੈ ਉਰਵਾਰਿ ਨ ਜਾਈ 
॥੧॥
 ਰਹਾਉ ॥ਰਾਤਿ ਜਿ ਸੋਵਹਿ ਦਿਨ ਕਰਹਿ ਕਾਮ ॥ਇਕੁ ਖਿਨੁ ਲੇਹਿ ਨ ਹਰਿ ਕੋ ਨਾਮ 
॥੨॥
ਹਾਥਿ ਤ ਡੋਰ ਮੁਖਿ ਖਾਇਓ ਤੰਬੋਰ ॥ਮਰਤੀ ਬਾਰ ਕਸਿ ਬਾਧਿਓ ਚੋਰ 
॥੩॥
ਗੁਰਮਤਿ ਰਸਿ ਰਸਿ ਹਰਿ ਗੁਨ ਗਾਵੈ ॥ਰਾਮੈ ਰਾਮ ਰਮਤ ਸੁਖੁ ਪਾਵੈ 
॥੪॥
ਕਿਰਪਾ ਕਰਿ ਕੈ ਨਾਮੁ ਦ੍ਰਿੜਾਈ ॥ਹਰਿ ਹਰਿ ਬਾਸੁ ਸੁਗੰਧ ਬਸਾਈ 
॥੫॥
ਕਹਤ ਕਬੀਰ ਚੇਤਿ ਰੇ ਅੰਧਾ ॥ਸਤਿ ਰਾਮੁ ਝੂਠਾ ਸਭੁ ਧੰਧਾ 
॥੬॥
੧੬॥


In [91]:
# Define the regex pattern
pattern = r'(\|\|\d+\|\|)'

# Split the Punjabi text based on the regex pattern
chunks = re.split(pattern, get_full_shabad(1300)["shabad_text_eng"])

# Print the resulting chunks
for chunk in chunks:
    print(chunk,'<')


Gauree, Kabeer Jee:You may anoint your limbs with sandalwood oil,but in the end, that body will be burned with the firewood.  <
||1|| <
Why should anyone take pride in this body or wealth?They shall end up lying on the ground; they shall not go along with you to the world beyond.  <
||1|| <
Pause||They sleep by night and work during the day,but they do not chant the Lord's Name, even for an instant.  <
||2|| <
They hold the string of the kite in their hands, and chew betel leaves in their mouths,but at the time of death, they shall be tied up tight, like thieves.  <
||3|| <
Through the Guru's Teachings, and immersed in His Love, sing the Glorious Praises of the Lord.Chant the Name of the Lord, Raam, Raam, and find peace.  <
||4|| <
In His Mercy, He implants the Naam within us;inhale deeply the sweet aroma and fragrance of the Lord, Har, Har.  <
||5|| <
Says Kabeer, remember Him, you blind fool!The Lord is True; all worldly affairs are false.  <
||6|| <
16|| <


In [53]:
def chunker_pa(shabad_pa):
    pattern = r'(॥[੦੧੨੩੪੫੬੭੮੯]+॥)'
    # Split the Punjabi text based on the regex pattern
    chunks = re.split(pattern, shabad_pa)
    return [chunk for chunk in chunks if len(chunk) >20]

def chunker_eng(shabad_eng):
    pattern = r'(\|\|\d+\|\|)'
    # Split the Punjabi text based on the regex pattern
    chunks = re.split(pattern, shabad_eng)
    return [chunk for chunk in chunks if len(chunk) >20]

In [54]:
chunks_pa = chunker_pa(full_shabad['shabad_text_pa'])

In [211]:
chunks_eng =chunker_eng(full_shabad['shabad_text_eng'])

In [43]:
def combine_chunks(chunks_pa, chunks_eng):
    combined_chunks = []
    if len(chunks_pa) == len(chunks_eng):
        for i_pa, i_eng in zip(chunks_pa, chunks_eng):
            combined_chunks.append(i_pa + " @ " + i_eng)
    else:
        print('mismatch between chunk lengths')
        return None
    return combined_chunks

In [55]:
# combine_chunks(chunks_pa, chunks_eng)

In [45]:
# get_embedding(combine_chunks(chunks_pa, chunks_eng)[0])

In [46]:
def pinecone_push_chunks(start,end, source):
    index_data = []
    index = pc.Index("bani-db-serverless")
    
    for i in range(start, end):
        full_shabad = get_full_shabad(i)
        shabad_pa = full_shabad['shabad_text_pa']
        # print(shabad_pa)
        chunks_pa = chunker_pa(shabad_pa)
        
        shabad_eng = full_shabad['shabad_text_eng']
        chunks_eng = chunker_eng(shabad_eng)
        combined_chunks = combine_chunks(chunks_pa, chunks_eng) #combine pa and eng chunks
        
        if not combined_chunks:
            print('skipping...'+ str(i) +' chunk length mismatch')
            # print(chunks_pa, chunks_eng)
            continue
        # combined chunks checks for matching lenghts. so it is ok to access indexs of chunks_pa and chunks_eng individually as well
        # print(len(combined_chunks),'len combined_chunks')
        for chunk_idx, chunk in enumerate(combined_chunks):
            # try:
                index.upsert(
                    vectors=[
                        {
                            "id": str(full_shabad['shabad_id']) + "." + str(chunk_idx),
                            "values": get_embedding(chunk[:8000]),   #8192 is the token limit, so skipp all after that in case limit is reached.
                            "metadata": {"shabad_pa": chunks_pa[chunk_idx],
                                        'shabad_id': full_shabad['shabad_id'],
                                        "shabad_eng": chunks_eng[chunk_idx],
                                         "shabad_combined": chunk,
                                        "source": source},
                        }
                    ],
                    namespace='pankti_combined')
                
                # if i%1==0:
                print("at index: ", i, ", pinecone id:",str(full_shabad['shabad_id']) + "." + str(chunk_idx))
            # except Exception as e:
            #     print('at index', i)
            #     print(e)


In [92]:
pinecone_push_chunks(1300,2000, "SGGS")

at index:  1300 , pinecone id: 1300.0
at index:  1300 , pinecone id: 1300.1
at index:  1300 , pinecone id: 1300.2
at index:  1300 , pinecone id: 1300.3
at index:  1300 , pinecone id: 1300.4
at index:  1300 , pinecone id: 1300.5
at index:  1300 , pinecone id: 1300.6
at index:  1301 , pinecone id: 1301.0
at index:  1301 , pinecone id: 1301.1
at index:  1301 , pinecone id: 1301.2
at index:  1301 , pinecone id: 1301.3
at index:  1302 , pinecone id: 1302.0
at index:  1302 , pinecone id: 1302.1
at index:  1302 , pinecone id: 1302.2
at index:  1302 , pinecone id: 1302.3
at index:  1302 , pinecone id: 1302.4
at index:  1303 , pinecone id: 1303.0
at index:  1303 , pinecone id: 1303.1
at index:  1303 , pinecone id: 1303.2
at index:  1303 , pinecone id: 1303.3
at index:  1304 , pinecone id: 1304.0
at index:  1304 , pinecone id: 1304.1
at index:  1304 , pinecone id: 1304.2
at index:  1304 , pinecone id: 1304.3
at index:  1305 , pinecone id: 1305.0
at index:  1305 , pinecone id: 1305.1
at index:  1

ProtocolError: Failed to connect; did you specify the correct index name?

In [229]:
1+1

2

In [15]:


### push 100 angs to pinecone
index_data = []
start=1000  # 1 based index
end=1005
index = pc.Index("bani-db-serverless")

for i in range(start, end):
  # blob = {}
    verse_text_pa = ""
    verse_text_eng = ""
    shabad = banidb.shabad(i, larivaar=False)
    for verse in shabad['verses']:
        verse_text_pa += verse['verse'] 
        verse_text_eng += verse["steek"]["en"]["bdb"]
      # blob['shabad_pa'] = shabad_text
      # blob['id'] =
      # index_data.append(blob)
        text_for_embedding = verse_text_pa + " " + verse_text_eng
        try:
            index.upsert(
                vectors=[
                {
                    "id": str(verse['verse_id']),
                    "values": get_embedding(text_for_embedding[:8000]),   #8192 is the token limit, so skipp all after that.
                    "metadata": {"shabad_pa": verse_text_pa,
                                'source_eng': shabad['source_eng'],
                                'writer': str(shabad['writer']),
                                'ang': shabad['ang'],
                                'shabad_id': shabad['shabad_id'],
                                "shabad_eng": verse_text_eng,
                                },

            }
        ],
        namespace='punjabi-and-english-verses-combined')
            if i%1==0:
                print("at index: ", i, verse['verse_id'])
        except Exception as e:
            print('at index', i)
            print(e)


at index:  1000 12702
at index:  1000 12703
at index:  1000 12704
at index:  1000 12705
at index:  1000 12706
at index:  1000 12707
at index:  1000 12708
at index:  1000 12709
at index:  1000 12710
at index:  1001 12711
at index:  1001 12712
at index:  1001 12713
at index:  1001 12714
at index:  1001 12715
at index:  1001 12716
at index:  1001 12717
at index:  1001 12718
at index:  1001 12719
at index:  1001 12720
at index:  1002 12721
at index:  1002 12722
at index:  1002 12723
at index:  1002 12724
at index:  1002 12725
at index:  1002 12726
at index:  1002 12727
at index:  1002 12728
at index:  1002 12729


KeyboardInterrupt: 

In [66]:
import numpy as np

In [84]:
index = pc.Index("bani-db-serverless")
xq = get_embedding("sefs")
xc = index.query(vector=list(np.random.random(1536)), top_k=2, include_metadata=True,namespace="pankti_combined")

In [85]:
# xq

In [86]:
# np.random.random(1536)

In [87]:
[match["metadata"] for match in xc['matches']]

[{'shabad_combined': 'ਹਉ ਵਾਰੀ ਜੀਉ ਵਾਰੀ ਨਿਰੰਕਾਰੀ ਨਾਮੁ ਧਿਆਵਣਿਆ ॥ਤਿਸੁ ਰੂਪੁ ਨ ਰੇਖਿਆ ਘਟਿ ਘਟਿ ਦੇਖਿਆ ਗੁਰਮੁਖਿ ਅਲਖੁ ਲਖਾਵਣਿਆ  @ I am a sacrifice, my soul is a sacrifice, to those who meditate on the Naam, the Name of the Formless Lord.He has no form or shape; He is seen within each and every heart. The Gurmukh comes to know the unknowable. ',
  'shabad_eng': 'I am a sacrifice, my soul is a sacrifice, to those who meditate on the Naam, the Name of the Formless Lord.He has no form or shape; He is seen within each and every heart. The Gurmukh comes to know the unknowable. ',
  'shabad_id': 353.0,
  'shabad_pa': 'ਹਉ ਵਾਰੀ ਜੀਉ ਵਾਰੀ ਨਿਰੰਕਾਰੀ ਨਾਮੁ ਧਿਆਵਣਿਆ ॥ਤਿਸੁ ਰੂਪੁ ਨ ਰੇਖਿਆ ਘਟਿ ਘਟਿ ਦੇਖਿਆ ਗੁਰਮੁਖਿ ਅਲਖੁ ਲਖਾਵਣਿਆ ',
  'source': 'SGGS'},
 {'shabad_combined': 'ਹਉ ਵਾਰੀ ਜੀਉ ਵਾਰੀ ਨਾਮੁ ਸੁਣਿ ਮੰਨਿ ਵਸਾਵਣਿਆ ॥ਹਰਿ ਜੀਉ ਸਚਾ ਊਚੋ ਊਚਾ ਹਉਮੈ ਮਾਰਿ ਮਿਲਾਵਣਿਆ  @ I am a sacrifice, my soul is a sacrifice, to those who hear and enshrine the Naam within their minds.The Dear Lord, the True One, the Highest of the High, subdues their eg

In [60]:
i=1
for shabad,embed in zip(shabads, embeddings):
  index.upsert(
      vectors=[
          {
              "id": str(i),
              "values": embed,
              "metadata": {"shabad": shabad}
          }
      ],
  )
  i+=1

NameError: name 'shabads' is not defined